In [2]:
import json
from pyspark import SparkConf,SparkContext

In [3]:
from pyspark import SparkConf,SparkContext
spark_conf = SparkConf().setMaster("local").setAppName("ReadHbase")
spark_context = SparkContext(conf = spark_conf)
# from pyspark.sql import SparkSession
# spark_session = SparkSession(spark_context)

In [4]:
hbase_host = "192.168.1.150"
hbase_table = "TEST_RPT"
hbase_conf = {"hbase.zookeeper.quorum": hbase_host,
              "hbase.mapreduce.inputtable": hbase_table,
              "hbase.mapreduce.scan.row.start": '18589802',
              "hbase.mapreduce.scan.row.stop" : '18589812',
              "hbase.mapreduce.scan.columns"  : "info:cont",
              # "hbase.mapreduce.scan.columns"  : "info:cont info:bd info:sex",
             }
              
keyConv = "org.apache.spark.examples.pythonconverters.ImmutableBytesWritableToStringConverter"
valueConv = "org.apache.spark.examples.pythonconverters.HBaseResultToStringConverter"

In [5]:
hbase_rdd = spark_context.newAPIHadoopRDD("org.apache.hadoop.hbase.mapreduce.TableInputFormat",
                                          "org.apache.hadoop.hbase.io.ImmutableBytesWritable",
                                          "org.apache.hadoop.hbase.client.Result",
                                          keyConverter=keyConv,
                                          valueConverter=valueConv,
                                          conf=hbase_conf,
                                          batchSize=0,
                                         )

KeyboardInterrupt: 

In [5]:
count = hbase_rdd.count()
count

3

In [6]:
def report_decode(hbase_string_data):
    decoded_data = json.loads(
        json.loads(
            hbase_string_data
        )['value'].encode().decode('unicode-escape').encode("raw_unicode_escape").decode()
    )
    return decoded_data

In [7]:
decode_rdd = hbase_rdd.map(lambda xxx:xxx[1])
decode_rdd = decode_rdd.map(lambda xxx:report_decode(xxx))
decode_rdd.cache()

PythonRDD[4] at RDD at PythonRDD.scala:48

In [8]:
print(decode_rdd.collect()[0])

{'checkItems': [{'checkItemName': '基础项目', 'checkResults': [{'appendInfo': '', 'canExplain': 0, 'checkIndexName': '收缩压', 'highValueRef': '', 'isAbandon': None, 'isCalc': None, 'lowValueRef': '', 'resultFlagId': 1, 'resultTypeId': None, 'resultValue': '148', 'showIndex': None, 'textRef': '', 'unit': 'mmHg'}, {'appendInfo': '', 'canExplain': 0, 'checkIndexName': '舒张压', 'highValueRef': '', 'isAbandon': None, 'isCalc': None, 'lowValueRef': '', 'resultFlagId': 1, 'resultTypeId': None, 'resultValue': '93', 'showIndex': None, 'textRef': '', 'unit': 'mmHg'}, {'appendInfo': '', 'canExplain': 1, 'checkIndexName': '身高', 'highValueRef': '', 'isAbandon': None, 'isCalc': None, 'lowValueRef': '', 'resultFlagId': 1, 'resultTypeId': None, 'resultValue': '161', 'showIndex': None, 'textRef': '', 'unit': 'Cm'}, {'appendInfo': '', 'canExplain': 1, 'checkIndexName': '体重', 'highValueRef': '', 'isAbandon': None, 'isCalc': None, 'lowValueRef': '', 'resultFlagId': 1, 'resultTypeId': None, 'resultValue': '73.6', 

In [9]:
generalSummarys_rdd = decode_rdd.map(lambda xxx: xxx['generalSummarys'])

In [1]:
from mod.summaryAnalyse_v4 import summary_analyse

ModuleNotFoundError: No module named 'mod'